In [1]:
import argparse
import os
import sys
import math
import torch
import torch.nn as nn
import pandas as pd

from thop import profile

import rl_harnas.genotypes as genotypes
from rl_harnas.model_generator import get_spec

from rl_harnas.nas_utils import AverageMeter

from data_providers import UCIHARDataProvider, OPPDataProvider, KUHARDataProvider, UniMiBDataProvider, WISDMDataProvider

# RLNAS

* cpu

In [1]:
for i in range(5):
    ! python rl_harnas/measure_latency.py --hardware pc --device cpu --dataset uci --arch RLNAS --num-runs 100 --data_path ./ --config_file harnas.csv 

RLNAS: 0.001778
Experiment results saved to harnas.csv
RLNAS: 0.001685
Experiment results saved to harnas.csv
RLNAS: 0.001830
Experiment results saved to harnas.csv
RLNAS: 0.001731
Experiment results saved to harnas.csv
RLNAS: 0.001774
Experiment results saved to harnas.csv


* gpu

In [2]:
for i in range(5):
    ! python rl_harnas/measure_latency.py --hardware pc --device gpu --dataset uci --arch RLNAS --num-runs 100 --data_path ./ --config_file harnas.csv 

RLNAS: 2.504682
Experiment results saved to harnas.csv
RLNAS: 2.188951
Experiment results saved to harnas.csv
RLNAS: 2.153836
Experiment results saved to harnas.csv
RLNAS: 2.163486
Experiment results saved to harnas.csv
RLNAS: 2.138327
Experiment results saved to harnas.csv


## Model Spec

In [4]:
! python rl_harnas/measure_metrics.py --dataset uci --arch RLNAS --config_file harnas_spec.csv

Flops: 249901376.000000, Param size: 2745574.000000MB
Flops: 354.363776, Param size: 2.745574MB
model size: 10.485MB
Experiment results saved to harnas_spec.csv


Unsupported operator aten::pad encountered 2 time(s)
Unsupported operator aten::lstm encountered 1 time(s)


# EANAS

* cpu

In [7]:
for i in range(5):
    ! python ea_harnas/measure_latency.py --hardware pc --device cpu --dataset uci --arch EANAS --num-runs 100 --data_path ./ --config_file harnas.csv 

18 48
EANAS: 0.001801
Experiment results saved to harnas.csv
18 48
EANAS: 0.001809
Experiment results saved to harnas.csv
18 48
EANAS: 0.001777
Experiment results saved to harnas.csv
18 48
EANAS: 0.001624
Experiment results saved to harnas.csv
18 48
EANAS: 0.001749
Experiment results saved to harnas.csv


* gpu

In [6]:
for i in range(5):
    ! python ea_harnas/measure_latency.py --hardware pc --device gpu --dataset uci --arch EANAS --num-runs 100 --data_path ./ --config_file harnas.csv 

18 48
EANAS: 2.512614
Experiment results saved to harnas.csv
18 48
EANAS: 2.433598
Experiment results saved to harnas.csv
18 48
EANAS: 2.493131
Experiment results saved to harnas.csv
18 48
EANAS: 2.432619
Experiment results saved to harnas.csv
18 48
EANAS: 2.425080
Experiment results saved to harnas.csv


## Model Spec

In [10]:
! python ea_harnas/measure_metrics.py --dataset uci --arch EANAS --config_file harnas_spec.csv

18 48
Flops: 19250688.000000, Param size: 729462.000000MB
Flops: 93.802496, Param size: 0.729462MB
model size: 2.786MB
Experiment results saved to harnas_spec.csv


Unsupported operator aten::max_pool2d encountered 2 time(s)
Unsupported operator aten::add encountered 4 time(s)
Unsupported operator aten::lstm encountered 1 time(s)


# Mobile-DNAS

* cpu

In [11]:
for i in range(5):
    ! python proposed_harnas/measure_latency.py --hardware pc --device cpu --dataset uci --arch OPPDNAS --num-runs 100 --data_path ./ --config_file harnas.csv 

6 48
OPPDNAS: 0.001794
Experiment results saved to harnas.csv
6 48
OPPDNAS: 0.001636
Experiment results saved to harnas.csv
6 48
OPPDNAS: 0.001782
Experiment results saved to harnas.csv
6 48
OPPDNAS: 0.001723
Experiment results saved to harnas.csv
6 48
OPPDNAS: 0.001781
Experiment results saved to harnas.csv


* gpu

In [12]:
for i in range(5):
    ! python proposed_harnas/measure_latency.py --hardware pc --device gpu --dataset uci --arch OPPDNAS --num-runs 100 --data_path ./ --config_file harnas.csv 

6 48
OPPDNAS: 1.831477
Experiment results saved to harnas.csv
6 48
OPPDNAS: 1.790599
Experiment results saved to harnas.csv
6 48
OPPDNAS: 1.850642
Experiment results saved to harnas.csv
6 48
OPPDNAS: 1.820622
Experiment results saved to harnas.csv
6 48
OPPDNAS: 1.788445
Experiment results saved to harnas.csv


## Model Spec

In [15]:
! python proposed_harnas/measure_metrics.py --dataset uci --arch OPPDNAS --config_file harnas_spec.csv

6 48
Flops: 7828224.000000, Param size: 358790.000000MB
Flops: 46.199552, Param size: 0.358790MB
model size: 1.372MB
Experiment results saved to harnas_spec.csv


Unsupported operator aten::avg_pool2d encountered 2 time(s)
Unsupported operator aten::add encountered 4 time(s)
Unsupported operator aten::lstm encountered 1 time(s)


# 결과 결합

In [19]:
import pandas as pd

In [20]:
lat_df = pd.read_csv('harnas.csv')
spec_df = pd.read_csv('harnas_spec.csv')

In [21]:
lat_df.head()

,dataset,batch,model,hardware,device,latency
0,uci,1,RLNAS,pc,cpu,0.001778
1,uci,1,RLNAS,pc,cpu,0.001685
2,uci,1,RLNAS,pc,cpu,0.001830
3,uci,1,RLNAS,pc,cpu,0.001731
4,uci,1,RLNAS,pc,cpu,0.001774


In [22]:
spec_df.head()

,dataset,Model,Params(M),FLOPs(M),ModelSize(MB)
0,uci,RLNAS,2.745574,354.363776,10.484741
1,uci,EANAS,0.729462,93.802496,2.785980
2,uci,OPPDNAS,0.358790,46.199552,1.371834


# calculate latency mean & std

mean

In [23]:
test = lat_df.groupby(['model', 'device', 'batch']).mean()
test

C:\Users\ai-ws\AppData\Local\Temp\ipykernel_37972\3156294001.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test = lat_df.groupby(['model', 'device', 'batch']).mean()


latency
model   device batch          
EANAS   cpu    1      0.001739
        gpu    1      2.459408
OPPDNAS cpu    1      0.001724
        gpu    1      1.816357
RLNAS   cpu    1      0.001760
        gpu    1      2.229857

In [24]:
test = test.reset_index()
test

,model,device,batch,latency
0,EANAS,cpu,1,0.001739
1,EANAS,gpu,1,2.459408
2,OPPDNAS,cpu,1,0.001724
3,OPPDNAS,gpu,1,1.816357
4,RLNAS,cpu,1,0.001760
5,RLNAS,gpu,1,2.229857


In [25]:
temp = test[test['device'] == 'cpu'].iloc[:, [0,2]].copy()
temp.reset_index(drop=True, inplace=True)

In [26]:
temp['PC:CPU(ms)'] = test[test['device'] == 'cpu']['latency'].reset_index(drop=True)
temp['PC:GPU(ms)'] = test[test['device'] == 'gpu']['latency'].reset_index(drop=True)
temp

,model,batch,PC:CPU(ms),PC:GPU(ms)
0,EANAS,1,0.001739,2.459408
1,OPPDNAS,1,0.001724,1.816357
2,RLNAS,1,0.001760,2.229857


In [27]:
temp.columns = ['Model', 'Batch', 'PC:CPU(ms)', 'PC:GPU(ms)']
temp

,Model,Batch,PC:CPU(ms),PC:GPU(ms)
0,EANAS,1,0.001739,2.459408
1,OPPDNAS,1,0.001724,1.816357
2,RLNAS,1,0.001760,2.229857


In [28]:
result_df = pd.merge(spec_df, temp, on='Model', how='left')
result_df

,dataset,Model,Params(M),FLOPs(M),ModelSize(MB),Batch,PC:CPU(ms),PC:GPU(ms)
0,uci,RLNAS,2.745574,354.363776,10.484741,1,0.001760,2.229857
1,uci,EANAS,0.729462,93.802496,2.785980,1,0.001739,2.459408
2,uci,OPPDNAS,0.358790,46.199552,1.371834,1,0.001724,1.816357


* save result of vision model

In [29]:
result_df.to_csv('result_harnas.csv', index=False)